## Linear Elasticity for Cantilever Beam

In [15]:
using Gridap, Gridap.CellData, SparseArrays
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics
using CSV, DataFrames

In [16]:
push!(LOAD_PATH, "utilities")
include("utilities/plot_design.jl")
include("utilities/OC.jl")
include("utilities/SIMP_Inter.jl")
include("utilities/filters.jl")
include("utilities/Surface_Vec.jl")
push!(LOAD_PATH, "FD_check")
include("FD_check/run_finite_difference_check.jl")

run_finite_difference_check (generic function with 1 method)

In [17]:
L = 20 # Length
W = 10 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (20, 10, 10) # mesh sizes
model = CartesianDiscreteModel(domain, partition);
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]); # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]); # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 

In [29]:
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "right")
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3,Float64}, order)
V = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags = "left")
U = TrialFESpace(V)
# Homogenized material properties
ν₀ = 0.3   # Poisson's ratio of based material
E₀ = 119e3 # Young's modulus of based material
G₀ = E₀ / (2 * (1 + 0.2)) # 0.2 is minimum \nu

ρ = 0.5    # density 
# Compute elastic constants using the provided formulas
Coeffs = Matrix(CSV.read("utilities/"*"Coeffs.csv", DataFrame, header=false))[1:3,:]
degree_x, degree_y = 3, 1
# Interpolation law
Coeffs_E, Coeffs_nu, Coeffs_G = Coeffs[1, :], Coeffs[2, :], Coeffs[3, :]
Pred_Vec, ∂_Pred_∂ρ = Surface_Vec(ρ, ν₀, degree_x, degree_y)
# =======================================================================
E = E₀ * dot(Pred_Vec ,Coeffs_E)
ν = dot(Pred_Vec, Coeffs_nu)
λ = (E*ν)/((1+ν)*(1-2*ν)) # C12
μ = G₀* dot(Pred_Vec, Coeffs_G)

∂E∂ρ = E₀ * dot(∂_Pred_∂ρ ,Coeffs_E)
∂λ∂ρ = (∂E∂ρ*ν)/((1+ν)*(1-2*ν)) 
∂μ∂ρ = G₀* dot(∂_Pred_∂ρ, Coeffs_G)
# -------------------------------
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀; # Stress
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # A unit load along Y axis applies to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,U,V)
uh = solve(op)


# Method 1
C1 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C1)

Compliance 1: 1.0910693971823717


In [ ]:
typeof()

In [19]:
δ = 1e-5
ρ = 0.5  + δ  # density 
# Compute elastic constants using the provided formulas
Coeffs = Matrix(CSV.read("utilities/"*"Coeffs.csv", DataFrame, header=false))[1:3,:]
degree_x, degree_y = 3, 1
# Interpolation law
Coeffs_E, Coeffs_nu, Coeffs_G = Coeffs[1, :], Coeffs[2, :], Coeffs[3, :]
Pred_Vec, ∂_Pred_∂ρ = Surface_Vec(ρ, ν₀, degree_x, degree_y)
# =======================================================================
E = E₀ * dot(Pred_Vec ,Coeffs_E)
ν = dot(Pred_Vec, Coeffs_nu)
λ = (E*ν)/((1+ν)*(1-2*ν)) # C12
μ = G₀* dot(Pred_Vec, Coeffs_G)

∂E∂ρ = E₀ * dot(∂_Pred_∂ρ ,Coeffs_E)
∂λ∂ρ = (∂E∂ρ*ν)/((1+ν)*(1-2*ν)) 
∂μ∂ρ = G₀* dot(∂_Pred_∂ρ, Coeffs_G)
# -------------------------------
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀; # Stress
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # A unit load along Y axis applies to each node on the right face

a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions

l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,U,V)
uh = solve(op)
# Method 1
C2 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C2)
dC_dρ_1 = (C1-C2)/ δ

Compliance 1: 1.0910353873837233


3.4009798648426677

In [20]:
∂σ∂ρ(ε) = ∂λ∂ρ * tr(ε) * one(ε) + 2 * ∂μ∂ρ * ε

dC_dρ = sum( ∫( (∂σ∂ρ ∘ ε₀(uh)) ⊙ ε₀(uh) ) * dΩ )

println("dC/dρ: ", dC_dρ)

dC/dρ: 3.4692731386696205


In [21]:
L = 20 # Length
W = 10 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (20, 10, 10) # mesh sizes
model = CartesianDiscreteModel(domain, partition);
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]); # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]); # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
Γ = BoundaryTriangulation(model)
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3,Float64}, order)

V = TestFESpace(Ω, reffe; conformity=:H1)
U = TrialFESpace(V)

RHS = assemble_vector(V) do v ∫(F ⋅ v) * dΓ end

LHS = assemble_matrix(U,V) do u,v  ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ end


7623×7623 SparseMatrixCSC{Float64, Int64} with 527589 stored entries:
⎡⠻⣦⡀⠘⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⣀⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢧⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢶⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠉⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⡄⠈⠻⣦⎦

In [23]:
# Compute centroids of elements and store in a matrix
cell_coords = get_cell_coordinates(Ω) # Get coordinates of vertices for each cell
cell_coords[1,1,10]

8-element Vector{VectorValue{3, Float64}}:
  VectorValue{3, Float64}(0.0, 0.0, 9.0)
  VectorValue{3, Float64}(1.0, 0.0, 9.0)
  VectorValue{3, Float64}(0.0, 1.0, 9.0)
  VectorValue{3, Float64}(1.0, 1.0, 9.0)
 VectorValue{3, Float64}(0.0, 0.0, 10.0)
 VectorValue{3, Float64}(1.0, 0.0, 10.0)
 VectorValue{3, Float64}(0.0, 1.0, 10.0)
 VectorValue{3, Float64}(1.0, 1.0, 10.0)

In [24]:
data=cell_coords[:]
flat_coords = reduce(vcat, data)
coors = hcat(map(x -> collect(x), Tuple.(flat_coords))...)



3×16000 Matrix{Float64}:
 0.0  1.0  0.0  1.0  0.0  1.0  0.0  …  19.0  20.0  19.0  20.0  19.0  20.0
 0.0  0.0  1.0  1.0  0.0  0.0  1.0     10.0  10.0   9.0   9.0  10.0  10.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0      9.0   9.0  10.0  10.0  10.0  10.0

In [25]:
# y-coordinates are in the 2nd row
y_vals = coors[2, :]

# Find indices where y == 0
idx_bottom = findall(==(minimum(y_vals)), y_vals)'
# Find indices where y == 0
idx_top = findall(==(maximum(y_vals)), y_vals)'

disp_region = vec([idx_bottom; idx_top])

disp_dirs = 1*ones(length(disp_region)) # Along x-axis

 disp_mag = zeros(1, length(disp_region))
    disp_dirs1 = ones(1, length(disp_region))
    disp_dirs2 = 2 * ones(1, length(disp_region))
    disp_dirs3 = 3 * ones(1, length(disp_region))

  disp_dirs = vec([vec(idx_bottom); vec(disp_dirs2); vec(disp_dirs3); vec(disp_dirs4)])
    disp_mag = vec([vec(disp_mag); vec(disp_mag); vec(disp_mag); vec(disp_mag2)])


UndefVarError: UndefVarError: `disp_dirs4` not defined

In [26]:
    FE_n_global_dof = size(RHS,1)
    FE_fixeddofs = vec(zeros(1, FE_n_global_dof))
    if FE_dim == 2
        FE_fixeddofs[Int.(2 * FE_BC.disp_node[FE_BC.disp_dof.==1] .- 1)] .= 1 # set prescribed x1 DOFs 
        FE_fixeddofs[Int.(2 * FE_BC.disp_node[FE_BC.disp_dof.==2])] .= 1   # set prescribed x2 DOFs 
    else
        FE_fixeddofs[Int.(3 * FE_BC.disp_node[FE_BC.disp_dof.==1] .- 2)] .= 1 # set prescribed x1 DOFs 
        FE_fixeddofs[Int.(3 * FE_BC.disp_node[FE_BC.disp_dof.==2] .- 1)] .= 1 # set prescribed x2 DOFs 
        FE_fixeddofs[Int.(3 * FE_BC.disp_node[FE_BC.disp_dof.==3])] .= 1   # set prescribed x3 DOFs 
    end
    FE_freedofs = Int.(.!(FE_fixeddofs .== 1))

    # by calling find() once to these functions to get the indices, the 
    # overhead of logical indexing may be removed.
    FE_fixeddofs_ind = findall(FE_fixeddofs .== 1)
    FE_freedofs_ind = findall(FE_freedofs .== 1)
    FE_n_free_dof = length(FE_freedofs) # the number of free DOFs


UndefVarError: UndefVarError: `FE_dim` not defined

In [27]:
DOF = get_cell_dof_ids(U) # Get coordinates of vertices for each cell
DOF[1]'

1×24 adjoint(::Vector{Int32}) with eltype Int32:
 1  4  64  67  694  697  757  760  2  5  …  3  6  66  69  696  699  759  762

In [28]:
size(RHS,1)

7623